In [194]:
from google.colab import files
uploaded = files.upload()

Saving te.csv to te.csv
Saving tr.csv to tr.csv


# **작업형1 **

Q1-1. age 컬럼의 3사분위수와 1사분위수의 차를 절대값으로 구하고, 소수점 버려서, 정수로 출력

In [131]:
import pandas as pd
df= pd.read_csv('data4-1.csv')
q3 = df['age'].quantile(0.75)
q1 = df['age'].quantile(0.25)
diff = int(abs(q3-q1))
diff


50

Q1-2.
- (loves반응+wows반응)/(reactions반응) 비율이
- 0.4보다 크고 0.5보다 작으면서,
- type 컬럼이 'video'인 데이터의 갯수


In [156]:
import pandas as pd
df= pd.read_csv('data4-2.csv')
df['new'] = (df['loves']+df['wows']) / df['reactions']
df['new']=df['new'].dropna()
ans = df.loc[(df.new >0.4)& (df.new<0.5) & (df['type']=='video')]
len(ans)

90

Q1-3.
-  date_added가 2018년 1월 이면서
- country가 United Kingdom 단독 제작인 데이터의 갯수

In [164]:
import pandas as pd
df= pd.read_csv('data4-3.csv')
import datetime
df['date_added'] = pd.to_datetime(df['date_added'])
ans = df.loc[(df['date_added'].dt.year==2018) & (df['date_added'].dt.month ==1.0) & (df.country =='United Kingdom') ]
len(ans)
# df['date_added'].dt.year
# df['date_added'].dt.month

6

# ** 작업형 2유형**

### [마케팅] 자동차 시장 세분화

- 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
- 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!

- 예측할 값(y): "Segmentation" (1,2,3,4)

- 평가: Macro f1-score

제출 형식:

```
ID,Segmentation
458989,1
458994,2
459000,3
459003,4
```

In [225]:
import pandas as pd
train = pd.read_csv('tr.csv')
test = pd.read_csv('te.csv')
# train.shape,test.shape
# train.isnull().sum()
# test.isnull().sum()
# test.describe()
# test.describe(include='O')
# train['Segmentation'].value_counts()
target = train.pop('Segmentation')

combined = pd.concat([train,test])
combined_oh = pd.get_dummies(combined)
train = combined_oh[:len(train)]
test = combined_oh[len(train):]


# cols = train.select_dtypes(include='O').columns
# from sklearn.preprocessing import LabelEncoder
# for col in cols:
#   le = LabelEncoder()
#   train[col] = le.fit_transform(train[col])
#   test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)
# X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth =7, n_estimators=300,random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val,pred,average='macro')

pred = rf.predict(test)
submit = pd.DataFrame({'ID, ':test['ID'],'Segmentation':pred})
submit.to_csv('result.csv',index=False)
pd.read_csv('result.csv').head(4)

,"ID,",Segmentation
0,458989,1
1,458994,3
2,459000,2
3,459003,3
